In [77]:
import pandas as pd

df_customers = pd.read_csv('./../datasets_limpios/olist_order_customer.csv')
df_orders = pd.read_csv('./../datasets_limpios/olist_orders_dataset.csv')
df_reviews = pd.read_csv('./../datasets_limpios/olist_order_reviews_dataset_clean.csv')

df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
 8   tiempo_entrega                 96476 non-null  object
dtypes: object(9)
memory usage: 6.8+ MB


In [78]:
df_reviews.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     99224 non-null  object
 4   review_comment_message   99224 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [79]:
df_customers.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [80]:
#Junto los dataset con inner para quedarme con las coincidencias solo, ya que necesito campos de todas las tablas para sacar las reviews por estado
# Me quedo solo con las columnas necesarias para lo que quiero:

df_merge = pd.merge(
    pd.merge(df_reviews, df_orders, on='order_id', how='inner'),
    df_customers, on='customer_id', how='inner'
)

print(df_merge.head(5))


df_merge.info()

                          review_id                          order_id  \
0  7bc2406110b926393aa56f80a40eba40  73fc7af87114b39712e6da79b0a377eb   
1  80e641a11e56f04c1ad469d5645fdfde  a548910a1c6147796b98fdf73dbeba33   
2  228ce5500dc1d8e020d8d1322874b6f0  f9e4b658b201a9f2ecdecbb34bed034b   
3  e64fb393e7b32834bb789ff8bb30750e  658677c97b385a9be170737859d3511b   
4  f7c4243c7fe1938f181bec41a392bdeb  8e6bfb81e283fa7e4f11123a3fb894f1   

   review_score review_comment_title  \
0             4             no title   
1             5             no title   
2             5             no title   
3             5             no title   
4             5             no title   

                              review_comment_message  \
0                                         no comment   
1                                         no comment   
2                                         no comment   
3              Recebi bem antes do prazo estipulado.   
4  Parabéns lojas lannister adorei compr

In [81]:
#Pongo los tipos de datos datetime para poder sacar una columna con la diferencia entre la fecha estimada y la fecha de entrega
def clean_datetime_columns_pandas(
    df: pd.DataFrame,
    datetime_params: dict = None
) -> pd.DataFrame:
    """
    Convierte a datetime SOLO las columnas listadas en datetime_params:
      datetime_params = { col_name: format_str, ... }
    No hay inferencia automática.
    """
    df = df.copy()
    datetime_params = datetime_params or {}

    for col, fmt in datetime_params.items():
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format=fmt, errors="coerce")
            print(f"*** Columna {col!r}: convertida con formato {fmt} ***")
    return df

datetime_params = {
        "review_creation_date": "%Y-%m-%dT%H:%M:%S.%f",
        "review_answer_timestamp": "%Y-%m-%dT%H:%M:%S.%f",
        "order_purchase_timestamp": "%Y-%m-%d %H:%M:%S",
        "order_approved_at": "%Y-%m-%d %H:%M:%S",
        "order_delivered_carrier_date": "%Y-%m-%d %H:%M:%S",
        "order_delivered_customer_date": "%Y-%m-%d %H:%M:%S",
        "order_estimated_delivery_date": "%Y-%m-%d",
    }

df_merge = clean_datetime_columns_pandas(df_merge, datetime_params)

df_merge.info()


*** Columna 'review_creation_date': convertida con formato %Y-%m-%dT%H:%M:%S.%f ***
*** Columna 'review_answer_timestamp': convertida con formato %Y-%m-%dT%H:%M:%S.%f ***
*** Columna 'order_purchase_timestamp': convertida con formato %Y-%m-%d %H:%M:%S ***
*** Columna 'order_approved_at': convertida con formato %Y-%m-%d %H:%M:%S ***
*** Columna 'order_delivered_carrier_date': convertida con formato %Y-%m-%d %H:%M:%S ***
*** Columna 'order_delivered_customer_date': convertida con formato %Y-%m-%d %H:%M:%S ***
*** Columna 'order_estimated_delivery_date': convertida con formato %Y-%m-%d ***
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   review_id                      99224 non-null  object        
 1   order_id                       99224 non-null  object        
 2   review_score            

In [82]:
df_merge['dias_retraso'] = (
    df_merge['order_delivered_customer_date'].dt.floor('D') - df_merge['order_estimated_delivery_date']
).dt.days.astype('Int64')

print(df_merge.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   review_id                      99224 non-null  object        
 1   order_id                       99224 non-null  object        
 2   review_score                   99224 non-null  int64         
 3   review_comment_title           99224 non-null  object        
 4   review_comment_message         99224 non-null  object        
 5   review_creation_date           99224 non-null  datetime64[ns]
 6   review_answer_timestamp        99224 non-null  datetime64[ns]
 7   customer_id                    99224 non-null  object        
 8   order_status                   99224 non-null  object        
 9   order_purchase_timestamp       99224 non-null  datetime64[ns]
 10  order_approved_at              99068 non-null  datetime64[ns]
 11  order_delivered

In [ ]:
df_customers = pd.read_csv('./../datasets_limpios/olist_order_customer.csv')

df_unique_customers = df_customers.groupby('customer_unique_id').filter(lambda x: len(x) == 1)
print(df_customers.shape)
a = df_customers[df_customers['customer_city'] == 'sao paulo']
print(a.shape)

print(df_unique_customers.shape)


a = df_unique_customers[df_unique_customers['customer_city'] == 'campinas']

print(a.shape)


bd = df_customers['customer_unique_id'].nunique()

print(bd)
df_customers.info()
df_customers

(99441, 5)
(15540, 5)
(93099, 5)
(1355, 5)
96096
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [90]:
df_sellers = pd.read_csv('./../datasets_limpios/olist_sellers_dataset.csv')
print(df_sellers)

                             seller_id  seller_zip_code_prefix  \
0     3442f8959a84dea7ee197c632cb2df15                   13023   
1     d1b65fc7debc3361ea86b5f14c68d2e2                   13844   
2     ce3ad9de960102d0677a81f5d0bb7b2d                   20031   
3     c0f3eea2e14555b6faeea3dd58c1b1c3                    4195   
4     51a04a8a6bdcb23deccc82b0b80742cf                   12914   
...                                ...                     ...   
3090  98dddbc4601dd4443ca174359b237166                   87111   
3091  f8201cab383e484733266d1906e2fdfa                   88137   
3092  74871d19219c7d518d0090283e03c137                    4650   
3093  e603cf3fec55f8697c9059638d6c8eb5                   96080   
3094  9e25199f6ef7e7c347120ff175652c3b                   12051   

            seller_city seller_state  
0              campinas           sp  
1            mogi guacu           sp  
2        rio de janeiro           rj  
3             sao paulo           sp  
4     bragan

In [84]:
duplicados_df = df_customers[df_customers.duplicated('customer_unique_id', keep=False)]

duplicados_df.sort_values(by='customer_unique_id')

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
35608,24b0e2bd287e47d54d193e7bbb51103f,00172711b30d52eea8b313a7f2cced02,45200,jequie,ba
19299,1afe8a9c67eec3516c09a8bdcc539090,00172711b30d52eea8b313a7f2cced02,45200,jequie,ba
20023,1b4a75b3478138e99902678254b260f4,004288347e5e88a27ded2bb23747066c,26220,nova iguacu,rj
22066,f6efe5d5c7b85e12355f9d5c3db46da2,004288347e5e88a27ded2bb23747066c,26220,nova iguacu,rj
72451,49cf243e0d353cd418ca77868e24a670,004b45ec5c64187465168251cd1c9c2f,57055,maceio,al
...,...,...,...,...,...
75057,1ae563fdfa500d150be6578066d83998,ff922bdd6bafcdf99cb90d7f39cea5b3,17340,barra bonita,sp
27992,bec0bf00ac5bee64ce8ef5283051a70c,ff922bdd6bafcdf99cb90d7f39cea5b3,17340,barra bonita,sp
79859,d064be88116eb8b958727aec4cf56a59,ff922bdd6bafcdf99cb90d7f39cea5b3,17340,barra bonita,sp
64323,4b231c90751c27521f7ee27ed2dc3b8f,ffe254cc039740e17dd15a5305035928,37640,extrema,mg
